In [61]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from matplotlib.pyplot import imshow
from IPython.display import Image, display
from shutil import copyfile

import os 
import sys

In [72]:
f = open('./ml_project/small_layer1.json', 'r')
layer1 = json.load(f)

f = open('./ml_project/small_layer2.json', 'r')
layer2 = json.load(f)

ingr_df = pd.read_json('ml_project/small_det_ingrs.json')
df1 = pd.read_json('./ml_project/small_layer1.json')
df2 = pd.read_json('./ml_project/small_layer2.json')
print(len(ingr_df), len(df1), len(df2))

3971 3971 3971


In [84]:
import os
from PIL import Image
import urllib.request
import urllib.parse
ORIG_PATH = ''
NEW_PATH = 'small_'

def filename_to_subdir(image_id):
    dirs = list(image_id[:4])
    return os.path.join(*dirs)


def get_image_feature(image, parent_dir):
    filename = image['id']
    sub_directory = filename_to_subdir(filename)

    directory = os.path.join(parent_dir, sub_directory)
    new_directory = NEW_PATH + directory
    filepath = os.path.join("ml_project", NEW_PATH+parent_dir, new_directory, filename)
    dir_filepath = os.path.join("ml_project", NEW_PATH+parent_dir, new_directory)
    try:
        f=open(filepath, 'r+b')
    except FileNotFoundError:
        if not os.path.exists(dir_filepath):
            os.makedirs(dir_filepath)
        try:
            urllib.request.urlretrieve(urllib.parse.quote(image['url']), filepath)
            f=open(filepath, 'r+b')
        except:
            print(image['url'])
            return None
    try:
        with Image.open(f) as img:
            img = resize_image(img, [256, 256]) #resize to 256,256
    except:
        return None

    pixels = np.asarray(img)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    return pixels.reshape((-1,))

def resize_image(image, size):
    """Resize an image to the given size."""
    return image.resize(size, Image.ANTIALIAS)



In [89]:
output = {"train":[], "val":[], "test":[]}

for index, row in df2.iterrows():
    if index % 100 ==0:
        print(index)
    recipe_id = row['recipe_id']
    images = row['images']
    try:
        valid = ingr_df[ingr_df['recipe_id']==recipe_id]['valid'].iloc[0]
        ingrs = ingr_df[ingr_df['recipe_id']==recipe_id]['ingredients'].iloc[0]
    except:
        continue
    ingrs_list = []
    for v, ing in zip(valid, ingrs):
        if v:
            ingrs_list.append(ing['text'])

    parent_dir = df1[df1.recipe_id==recipe_id]['partition'].to_string(index=False)
    parent_dir = parent_dir.replace(' ', '')
    image_feature_list = []
    if len(images)>0:
        for image in images:
            feat = get_image_feature(image, parent_dir)
            image_feature_list.append(feat)
    else:
        continue

    if len(image_feature_list) > 0:
        output[parent_dir].append((recipe_id, image_feature_list, ingrs_list))
    

ttp://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/82/56/2/piczHtUyB.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/82/56/2/picnGb657.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/82/56/2/pici3ZzQh.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/48/15/88/picsKdlOQ.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/85/05/2/picVC9wbp.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/85/05/2/picA2cTFc.jpg
https://img-global.cpcdn.com/001_recipes/5352046250164224/0x0/photo.jpg
http://tastykitchen.com/recipes/wp-content/uploads/sites/2/2012/06/Black-Bottom-Cookie-Bars-410x273.jpg
http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/12/25/15/pick2ZCZg.jpg
http://img.sndimg.

In [91]:
print(len(output['train']), len(output['val']), len(output['test']))

2781 618 572


In [93]:
import pickle
pickle.dump(output, open("extracted_data.pkl", 'wb'))
